In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


 ## toilettes PMR  OK
filtrés sur :
*   acces PMR

colonnes drop :
*   relais_bebe (peu de pertinence)
*   url_fiche equipement
*   statut (je n'ai pas gardé car la doc ne dit pas a quoi sa corespond)

j'ai gardé l'url de l'api

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/A-K_thon/original_files/sanisettesparis.csv',sep=';')
# api = https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/sanisettesparis/records?limit=20

In [ ]:
df_EPMR = df[df['ACCES_PMR'] == 'Oui']

In [ ]:
list_sup = ['RELAIS_BEBE','URL_FICHE_EQUIPEMENT','STATUT','ACCES_PMR']

In [ ]:
df_EPMR = df_EPMR.drop(columns = list_sup)

In [ ]:
df_EPMR.rename(columns={'geo_point_2d': 'geo'}, inplace=True)

In [ ]:
df_EPMR = df_EPMR[df_EPMR['ARRONDISSEMENT'].between(75001, 75020)]

df_EPMR['ARRONDISSEMENT'] = 'PARIS ' + (filtered_df['ARRONDISSEMENT'] - 75000).astype(str) + 'EME ARRONDISSEMENT'

In [ ]:
df_EPMR.to_csv('/content/drive/MyDrive/df_EPMR.csv', index=False)

## ascenseur PMR OK

filtré sur EPMR voie publique => reste 2 lignes au final.

In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/A-K_thon/original_files/ascenseurs-escalators-tele-surveillance-temps-reel.csv',sep=';')
print(df2.to_markdown())

|    |   Identifiant télésurveillance | Equipement Télésurveillé   | Identifiant   | Equipement                    |   Identifiant réf-équipement | Nom équipement          | Type équipement         |   Arrondissement | Adresse                            |   Code Postal | Ville   | Complement localisation   | Coordonnées géograhiques               | Heure ouverture                          | Statut équipement   | Horodatage statut         |
|---:|-------------------------------:|:---------------------------|:--------------|:------------------------------|-----------------------------:|:------------------------|:------------------------|-----------------:|:-----------------------------------|--------------:|:--------|:--------------------------|:---------------------------------------|:-----------------------------------------|:--------------------|:--------------------------|
|  0 |                            nan | Non télé-surveillé         | 12807         | VIADUC DES ARTS            

In [ ]:
df_ASC = df2[df2['Nom équipement'] == 'EPMR VOIE PUBLIQUE']

In [ ]:
list_sup2 = ['Identifiant télésurveillance','Equipement Télésurveillé','Identifiant','Identifiant réf-équipement','Nom équipement','Complement localisation','Statut équipement','Horodatage statut']
df_ASC = df_ASC.drop(columns = list_sup2)

In [ ]:
display(df_ASC)

,Equipement,Type équipement,Arrondissement,Adresse,Code Postal,Ville,Coordonnées géograhiques,Heure ouverture
10,SQUARE MARCEL RAMJAN,Ascenseur voie publique,11,15 RUE MERLIN,75011,PARIS,"48.86048516712044, 2.384902238845825",H24
26,JARDIN BERGES DE LA SEINE,Ascenseur voie publique,7,2 RUE DU GROS CAILLOU,75007,PARIS,"48.863133558109624, 2.30454359203577",Aux horaires d’ouverture de l’équipement


In [ ]:
df_ASC.rename(columns={'Coordonnées géograhiques': 'geo'}, inplace=True)

In [ ]:
df_ASC.to_csv('/content/drive/MyDrive/df_ASC.csv', index=False)

## etablissements tourisme handicap OK
filtré sur la ville de paris
colonnes drop :
*   ['FAX','REFERENCE','Handicap mental','Handicap auditif','Handicap visuel','Handicap moteur','DEPARTEMENT','REGION','RESEAU','VILLE']



In [ ]:
df_ETH = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cartographie_des_etablissements_tourisme_handicap.csv',sep=';')
# api = https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/fontaines-a-boire/records?limit=20

In [ ]:
df_ETH['TELEPHONE'] = df_ETH['TELEPHONE'].astype(str).str.zfill(10)

In [ ]:
df_ETH = df_ETH[df_ETH['VILLE'] == 'PARIS']

In [ ]:
list_sup2 = ['FAX','REFERENCE','Handicap mental','Handicap auditif','Handicap visuel','Handicap moteur','DEPARTEMENT','REGION','RESEAU','VILLE']
df_ETH = df_ETH.drop(columns = list_sup2)

In [ ]:
keyword_to_activity = {
    'Jardins': 'Parc',
    'Pavillon': 'Parc',
    'Village': 'Parc',
    'Musée': 'Lieu de visite',
    'Touristique': 'Lieu de visite',
    'art': 'Lieu de visite',
    'spectacle': 'Lieu de visite',
    'Jardin': 'Parc',
    'Muséum': 'Lieu de visite'
}

def replace_activit(row):
  if pd.isna(row['ACTIVIT']):
    for keyword, activity in keyword_to_activity.items():
      if keyword in row['ETABLISSEMENT']:
        return activity
  return row['ACTIVIT']

df_ETH['ACTIVIT'] = df_ETH.apply(replace_activit, axis=1)

In [ ]:
df_ETH.rename(columns={'geo_point_2d': 'geo'}, inplace=True)

In [ ]:
df_ETH = df_ETH[df_ETH['CODEPOSTAL'].between(75001, 75020)]

df_ETH['ARRONDISSEMENT'] = 'PARIS ' + (df_ETH['CODEPOSTAL'] - 75000).astype(str) + 'EME ARRONDISSEMENT'

In [ ]:
df_ETH = df_ETH.drop(columns = 'CODEPOSTAL')

In [ ]:
df_ETH.to_csv('/content/drive/MyDrive/df_ETH.csv', index=False)

## fontaines a boire

In [ ]:
df_FONT = pd.read_csv('/content/drive/MyDrive/A-K_thon/original_files/fontaines-a-boire.csv',sep=';')

In [ ]:
df_FONT['Adresse'] = df_FONT['N° Voie Pair'].fillna('') + ' ' + \
                     df_FONT['N° Voie Impair'].fillna('') + ' ' + \
                     df_FONT['Voie'].fillna('')

df_FONT['Adresse'] = df_FONT['Adresse'].str.strip()

In [ ]:
a_sup = ['Identifiant','Début Indisponibilité','Fin Indisponibilité','Motif Indisponibilité','Disponibilité','Modèle','N° Voie Pair','N° Voie Impair','Voie']
df_FONT = df_FONT.drop(columns = a_sup)

In [ ]:
df_FONT = df_FONT[df_FONT['Commune'].str.contains("PARIS", case=False)]

In [ ]:
df_FONT.rename(columns={'geo_point_2d': 'geo'}, inplace=True)

In [ ]:
df_FONT.to_csv('/content/drive/MyDrive/df_FONT.csv', index=False)

## merge

In [ ]:
df_FONT.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1077 entries, 0 to 1236
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Type Objet  1077 non-null   object
 1   Commune     1077 non-null   object
 2   geo_shape   1077 non-null   object
 3   geo         1077 non-null   object
 4   Adresse     1077 non-null   object
dtypes: object(5)
memory usage: 50.5+ KB


In [ ]:
display(df_FONT.head())

,Type Objet,Commune,geo_shape,geo,Adresse
0,BORNE_FONTAINE,PARIS 12EME ARRONDISSEMENT,"{""coordinates"": [2.406058144587103, 48.8473440...","48.847344001336964, 2.406058144587103",80F COURS DE VINCENNES
1,FONTAINE_2EN1,PARIS 20EME ARRONDISSEMENT,"{""coordinates"": [2.400335192859108, 48.8641063...","48.86410634144699, 2.400335192859108",6F PLACE EMILE LANDRIN
2,FONTAINE_ARCEAU,PARIS 2EME ARRONDISSEMENT,"{""coordinates"": [2.345972250830021, 48.8647428...","48.86474285287837, 2.345972250830021",60 RUE TIQUETONNE
3,FONTAINE_ARCEAU,PARIS 11EME ARRONDISSEMENT,"{""coordinates"": [2.3703489691991453, 48.853518...","48.85351841071337, 2.3703489691991453",6F RUE DE LA ROQUETTE
4,FONTAINE_BOIS,PARIS 12EME ARRONDISSEMENT,"{""coordinates"": [2.4418003364380603, 48.844865...","48.84486597992987, 2.4418003364380603",ROUTE DE LA DAME BLANCHE
